In [ ]:
# Cloud Shell
gcloud auth list

In [ ]:
export IAP_NET_TAG=grant-ssh-iap-ingress-ql-605
export INT_NET_TAG=grant-ssh-internal-ingress-ql-605
export HTTP_NET_TAG=grant-http-ingress-ql-605
export ZONE=us-east4-a

1. Check the firewall rules. Remove the overly permissive rules.

In [ ]:
gcloud compute firewall-rules delete open-access --quiet

2. Navigate to Compute Engine in the Cloud console and identify the bastion host. The instance should be stopped. Start the instance.

In [ ]:
gcloud compute instances start bastion \
    --project=$DEVSHELL_PROJECT_ID \
    --zone=$ZONE

3. The bastion host is the one machine authorized to receive external SSH traffic. Create a firewall rule that allows SSH (tcp/22) from the IAP service. The firewall rule must be enabled for the bastion host instance using a network tag of grant-ssh-iap-ingress-ql-605.

In [ ]:
gcloud compute firewall-rules create ssh-ingress --allow=tcp:22 --source-ranges 35.235.240.0/20 --target-tags $IAP_NET_TAG --network acme-vpc

In [ ]:
gcloud compute instances add-tags bastion --tags=$IAP_NET_TAG --zone=$ZONE

4. The juice-shop server serves HTTP traffic. Create a firewall rule that allows traffic on HTTP (tcp/80) to any address. The firewall rule must be enabled for the juice-shop instance using a network tag of grant-http-ingress-ql-605.

In [ ]:
gcloud compute firewall-rules create http-ingress --allow=tcp:80 --source-ranges 0.0.0.0/0 --target-tags $HTTP_NET_TAG --network acme-vpc

In [ ]:
gcloud compute instances add-tags juice-shop --tags=$HTTP_NET_TAG --zone=$ZONE

5. You need to connect to juice-shop from the bastion using SSH. Create a firewall rule that allows traffic on SSH (tcp/22) from acme-mgmt-subnet network address. The firewall rule must be enabled for the juice-shop instance using a network tag of grant-ssh-internal-ingress-ql-605.

In [ ]:
gcloud compute firewall-rules create internal-ssh-ingress --allow=tcp:22 --source-ranges 192.168.10.0/24 --target-tags $INT_NET_TAG --network acme-vpc

In [ ]:
gcloud compute instances add-tags juice-shop --tags=$INT_NET_TAG --zone=$ZONE

In [ ]:
cat > prepare_disk.sh <<'EOF_END'

export ZONE=$(gcloud compute instances list juice-shop --format 'csv[no-heading](zone)')

gcloud compute ssh juice-shop --internal-ip --zone=$ZONE --quiet

EOF_END

In [ ]:
gcloud compute scp prepare_disk.sh bastion:/tmp --project=$DEVSHELL_PROJECT_ID --zone=$ZONE --quiet

6. In the Compute Engine instances page, click the SSH button for the bastion host. Once connected, SSH to juice-shop.

In [ ]:
gcloud compute ssh bastion --project=$DEVSHELL_PROJECT_ID --zone=$ZONE --quiet --command="bash /tmp/prepare_disk.sh"